Вам нужно написать код на python, который сделает дедубликацию торговых точек. Вы можете использовать свой алгоритм и/или сторонние(внешние) сервисы.
Пожалуйста, предоставьте ссылку на репозиторий Github(Bitbucket), где будет находиться Ваш код и конечный результат(SQL таблицы с дедублицированными данными).

Вам нужно будет заполнить таблицу “outlets_clean” и обновить внешний ключ в таблице “outlets”(колонка outlet_clean_id). Для точек в таблице “outlets”, которые Вы не можете дедублицировать, проставьте в колонке outlet_clean_id значение NULL.


In [69]:
# import jellyfish as jf
# import numpy as np

import mysql.connector
import pandas as pd
import random
from sqlalchemy import create_engine
from thefuzz import fuzz
import time
from tqdm import tqdm

from config import USER, HOST, PASS, DB_NAME
from eidos import NameProcessing

In [2]:
connection = mysql.connector.connect(host=HOST, user=USER, passwd=PASS, db=DB_NAME)
outlets = pd.read_sql_query("SELECT * FROM `outlets`", connection)
outlets_clean = pd.read_sql_query("SELECT * FROM `outlets_clean`", connection)
connection.close()

In [38]:
# Создают случайный код в формате XXXX-XXXX-XXXX-XXXX
def cluster_id():
    id = ''

    for x in range(16):
        id = id + random.choice(list('1234567890abcdefghigklmnopqrstuvyxwzABCDEFGHIGKLMNOPQRSTUVYXWZ'))
    
        if len(id) == 4:
            id += '-'
    
        if len(id) == 9:
            id += '-'
        
        if len(id) == 14:
            id += '-'

    return id

# Функция для обработки DataFrame объекта
def DataProcessing(data, counter=1, read_from='Торг_точка_грязная', write_to='cluster_id', method=fuzz.ratio): 
    counter_f = counter
    for i in tqdm(data[read_from]):
        s = NameProcessing(i)
        k = s.cleaned_string
        if data[write_to].loc[counter_f] == 0:
            code = cluster_id()
            for j in range(len(data[read_from])):
                n = NameProcessing(data[read_from][j+counter])
                m = n.cleaned_string
                if method(k, m) == 100:
                    data[write_to].loc[j+counter] = code
            counter_f += 1
        else:
            counter_f += 1

# Функция для обработки DataFrame объекта
def SubclusterProcessing(data, counter=1, read_from='Торг_точка_грязная', write_to='subcluster_id', method=fuzz.WRatio):
    counter_f = counter
    for i in data[read_from]:
        s1 = data[read_from][counter_f]
        if data[write_to].loc[counter_f] == 0:
            code = cluster_id()
            for j in range(len(data[read_from])):
                s2 = data[read_from][j+counter]
                if method(s1, s2) > 95:
                    new_df[write_to].loc[data['id'].loc[j+counter]] = code
            counter_f += 1
        else:
            counter_f += 1

In [4]:
outlets = outlets.set_index('id')
new_df = outlets
new_df['cluster_id'] = 0
new_df.head(10)

,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,cluster_id
id,,,,,
1,Ростов-на-Дону,ИП Гаврилов С.А.,"п.Глубокий, ул.Артёма 44",None,0
2,Ростов-на-Дону,ИП Малышева,-,None,0
3,Ростов-на-Дону,Абасян А.П. ИП,он же,None,0
4,Ростов-на-Дону,"Азатян Н,С. ИП ул. Ленина",он же,None,0
5,Ростов-на-Дону,Айдинов В.Я.,он же,None,0
6,Ростов-на-Дону,Алексеева Л П ИП,Буденного 161,None,0
7,Ростов-на-Дону,Алексеева Л П ИП,он же,None,0
8,Ростов-на-Дону,ИП Алиева Т.М.,Ип Алиева Т.М. п.Газетный 39,None,0
9,Ростов-на-Дону,Алпатова В В ИП,он же,None,0


In [5]:
# Время выполнения 35 минут
DataProcessing(new_df, counter=1, read_from='Торг_точка_грязная', write_to='cluster_id', method=fuzz.ratio)

  0%|          | 0/20208 [00:00<?, ?it/s]C:\Users\deuse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 20208/20208 [34:40<00:00,  9.71it/s] 


In [68]:
new_df.groupby('cluster_id').size().sort_values(ascending=False).head(10)

cluster_id
VfGT-xYix-MOVm-fIxG    220
pVap-AoCY-SgKA-9upD    145
RYIQ-vTFf-0CeP-G80H     89
zSce-UNzz-nIEk-h7MC     81
Rk0r-r1RI-FzyU-u53R     77
1f4A-Fuha-Coo5-xOxd     70
2SaP-040m-Om2c-YTym     63
zmqq-xhDG-vg4H-4HNH     57
nGXm-Pq6h-OyTQ-Yufe     56
DOoe-ipaL-SmoR-lTw3     52
dtype: int64

In [78]:
new_df[new_df['cluster_id']=='gZpr-FwnO-XkMg-xKMp']

,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,cluster_id,subcluster_id
id,,,,,,
9,Ростов-на-Дону,Алпатова В В ИП,он же,None,gZpr-FwnO-XkMg-xKMp,ElZe-Dez0-bapV-Wuwh
10,Ростов-на-Дону,Алпатова В В ИП,ул Гвардейская 34-а,None,gZpr-FwnO-XkMg-xKMp,ElZe-Dez0-bapV-Wuwh
11,Ростов-на-Дону,Алпатова В.В. ИП,он же,None,gZpr-FwnO-XkMg-xKMp,ElZe-Dez0-bapV-Wuwh
12,Ростов-на-Дону,"Алпатова В.В. ИП ""Ассорти+""",он же,None,gZpr-FwnO-XkMg-xKMp,1gAp-Flhl-5gMA-4OiF
13,Ростов-на-Дону,Алпатова В.В. ИП ул.Гвардейская 34-а,он же,None,gZpr-FwnO-XkMg-xKMp,ICsu-ZdiC-UOLG-KM3g
11406,Ростов-на-Дону,Алпатова В.В. ИП,"346335, Ростовская обл, Донецк г, Пролетарская...",None,gZpr-FwnO-XkMg-xKMp,ElZe-Dez0-bapV-Wuwh
11407,Ростов-на-Дону,"Алпатова В.В. ИП ""Ассорти+""",-,None,gZpr-FwnO-XkMg-xKMp,1gAp-Flhl-5gMA-4OiF
11408,Ростов-на-Дону,"Алпатова В.В. ИП ""Ассорти+""","346335, Ростовская обл, Донецк г, Пролетарская...",None,gZpr-FwnO-XkMg-xKMp,1gAp-Flhl-5gMA-4OiF
11409,Ростов-на-Дону,Алпатова В.В. ИП ул.Гвардейская 34-а,-,None,gZpr-FwnO-XkMg-xKMp,ICsu-ZdiC-UOLG-KM3g


In [49]:
new_df['subcluster_id'] = 0

In [43]:
main_cluster_list = []
for c in new_df['cluster_id']:
    main_cluster_list.append(c)

main_cluster_list = list(set(main_cluster_list))
len(main_cluster_list)

7064

In [51]:
for cluster in tqdm(main_cluster_list):
    df = new_df[new_df['cluster_id']==cluster].reset_index()
    SubclusterProcessing(df, counter=0, read_from='Торг_точка_грязная', write_to='subcluster_id', method=fuzz.WRatio)
    time.sleep(0.2) # При желании либо убрать либо сделать 0.1

  0%|          | 0/7064 [00:00<?, ?it/s]C:\Users\deuse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 7064/7064 [24:49<00:00,  4.74it/s]


In [58]:
sub_cluster_list = []
for sc in new_df['subcluster_id']:
    sub_cluster_list.append(sc)

sub_cluster_list = list(set(sub_cluster_list))
print(f'sub_cluster_list_len: {len(sub_cluster_list)}')

sub_cluster_list_len: 9997


In [80]:
new_df[new_df['cluster_id']=='9w17-piCM-z0k0-vXs8'] #.groupby('subcluster_id').size().sort_values(ascending=False)

,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,cluster_id,subcluster_id
id,,,,,,
4547,Ростов-на-Дону,ТЕРРА ООО \339 Стр. Д.,-,None,9w17-piCM-z0k0-vXs8,pqz1-OULp-FAqH-O6tX
4548,Ростов-на-Дону,ТЕРРА ООО \339 Стр. Д.,"г. Ростов-на-Дону, ул. Жмайлова, 4/5",None,9w17-piCM-z0k0-vXs8,pqz1-OULp-FAqH-O6tX
4549,Ростов-на-Дону,ТЕРРА ООО \339 Стр Д,г Ростов-на-Дону ул Жмайлова 4/5,None,9w17-piCM-z0k0-vXs8,pqz1-OULp-FAqH-O6tX
4550,Ростов-на-Дону,ТЕРРА ООО \339 Стр. Д.,г. Ростов-на-Дону ул. Жмайлова 4/5,None,9w17-piCM-z0k0-vXs8,pqz1-OULp-FAqH-O6tX
4551,Ростов-на-Дону,ТЕРРА ООО /339 Стр. Д. / бонус,-,None,9w17-piCM-z0k0-vXs8,AvGX-paim-nnLB-C9sP
4552,Ростов-на-Дону,ТЕРРА ООО /Краснодарская,-,None,9w17-piCM-z0k0-vXs8,gWso-XIs7-FFou-re4z
4553,Ростов-на-Дону,ТЕРРА ООО /Краснодарская,г Ростов-на-Дону ул Краснодарская 143/2,None,9w17-piCM-z0k0-vXs8,gWso-XIs7-FFou-re4z
4554,Ростов-на-Дону,ТЕРРА ООО /Краснодарская,г.Ростов-на-Дону ул.Краснодарская 143/2,None,9w17-piCM-z0k0-vXs8,gWso-XIs7-FFou-re4z
4555,Ростов-на-Дону,ТЕРРА ООО /Краснодарская,г.Ростов-на-Дону ул.Краснодарская 143/2,None,9w17-piCM-z0k0-vXs8,gWso-XIs7-FFou-re4z


In [73]:
# Отправляем таблицу в нашу БД
engine = create_engine(f"mysql+mysqldb://{USER}:{PASS}@{HOST}/{DB_NAME}")
new_df.to_sql('new_df', con = engine)

In [83]:
# Делаем дам в CSV
new_df.to_csv('new_df.csv', sep=';')